In [268]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

%matplotlib inline

import numpy as np
import pandas as pd
import urllib3

from sklearn.preprocessing import MinMaxScaler

from lxml import html
from lxml import etree
import requests
import concurrent.futures
import time
import csv
import pickle

from urllib.parse import urlparse
from urllib.parse import parse_qs

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [269]:
dealers_df = pd.read_csv('task_3_data/dealers.csv')

dealers_df = dealers_df.rename(columns={"Unnamed: 0": "dealer_id", "0": "dealer_name"})

dealers_df['dealer_id'] = dealers_df['dealer_id'].astype('Int64').astype(str)

dealers_df.head()

,dealer_id,dealer_name
0,4064,#1 SG Car Deals Pte Ltd
1,3194,1 & Ones Motoring Pte Ltd
2,2556,1 Auto Co Pte Ltd
3,3843,1 Stop Motoring Pte Ltd
4,3965,111 Motoring Pte Ltd


In [270]:
dealer_listings_df = pd.read_csv('task_3_data/dealer_listings.csv')

dealer_listings_df['dealer_id'] = dealer_listings_df['dealer_id'].astype('Int64').astype(str)

dealer_listings_df.head()

,Unnamed: 0,dealer_id,listing_id,name,price,depreciation,reg_date,engine_cap,mileage,vehicle_type,status
0,0,3844,1028560,Volvo S60 T4 R-Design,NaN,NaN,10-Sep-2020,1969.0,NaN,Luxury,SOLD
1,1,3844,1021502,Mercedes-Benz C-Class C180,NaN,NaN,27-Feb-2014,1595.0,NaN,Luxury,SOLD
2,2,3844,1023851,Honda Fit 1.3A G Skyroof (COE till 08/2023),NaN,NaN,04-Aug-2008,1339.0,NaN,Hatchback,SOLD
3,3,4016,1019801,Mercedes-Benz C-Class C200K (COE till 03/2029),64900.0,8770.0,15-Apr-2009,1796.0,136000.0,Luxury,Available
4,4,4016,1026210,Honda Stream 1.8A X (COE till 09/2023),12800.0,7040.0,04-Sep-2008,1799.0,240000.0,MPV,Available


In [271]:
len(set(dealer_listings_df['listing_id']))

31816

In [272]:
df_task1_train_prediction = pd.read_csv('task_3_data/train_with_predicted_price_and_dealer.csv')

df_task1_train_prediction = df_task1_train_prediction.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, errors='ignore')

df_task1_train_prediction['dealer_id'] = df_task1_train_prediction['dealer_id'].astype('Int64').astype(str)

dealer_listings_df['listing_id'] = dealer_listings_df['listing_id'].astype(str)

len(set(df_task1_train_prediction['dealer_id']))

986

In [273]:
df_task1_train_prediction.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price,predicted_price,dealer_id
0,1030324,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013.0,NaN,09-dec-2013,luxury sedan,"parf car, premium ad car, low mileage car",auto,1560.0,135.0,NaN,1997.0,1.0,17700.0,77100.0,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,NaN,uncategorized,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",NaN,71300.0,97407.978598,2389
1,1021510,Toyota Hiace 3.0M,NaN,hiace,high loan available! low mileage unit. wear an...,2014.0,NaN,26-jan-2015,van,premium ad car,manual,1740.0,NaN,diesel,2982.0,3.0,11630.0,10660.0,NaN,3648.0,110112.0,27502.0,1376.0,NaN,25-jan-2035,uncategorized,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,NaN,43800.0,64184.991920,2976
2,1026909,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016.0,NaN,25-jul-2016,luxury sedan,"parf car, premium ad car",auto,1430.0,90.0,NaN,1595.0,1.0,15070.0,53694.0,740.0,44517.0,80000.0,27886.0,26041.0,NaN,NaN,uncategorized,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,NaN,95500.0,84798.727152,2217
3,1019371,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019.0,NaN,17-nov-2020,luxury sedan,"parf car, almost new car, consignment car",auto,1635.0,115.0,NaN,1497.0,1.0,16400.0,40690.0,684.0,80301.0,9800.0,46412.0,56977.0,NaN,NaN,uncategorized,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,NaN,197900.0,199098.750383,3734
4,1031014,Honda Civic 1.6A VTi,NaN,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019.0,NaN,20-sep-2019,mid-sized sedan,parf car,auto,1237.0,92.0,NaN,1597.0,1.0,10450.0,26667.0,742.0,36453.0,40000.0,20072.0,20101.0,NaN,NaN,uncategorized,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",NaN,103200.0,92988.269468,2202


In [274]:
print(len(set(dealer_listings_df['dealer_id']).intersection(df_task1_train_prediction['dealer_id'])))

868


In [275]:
dealer_listings_df_clone = dealer_listings_df.copy()

dealer_listings_df_clone.head()

,Unnamed: 0,dealer_id,listing_id,name,price,depreciation,reg_date,engine_cap,mileage,vehicle_type,status
0,0,3844,1028560,Volvo S60 T4 R-Design,NaN,NaN,10-Sep-2020,1969.0,NaN,Luxury,SOLD
1,1,3844,1021502,Mercedes-Benz C-Class C180,NaN,NaN,27-Feb-2014,1595.0,NaN,Luxury,SOLD
2,2,3844,1023851,Honda Fit 1.3A G Skyroof (COE till 08/2023),NaN,NaN,04-Aug-2008,1339.0,NaN,Hatchback,SOLD
3,3,4016,1019801,Mercedes-Benz C-Class C200K (COE till 03/2029),64900.0,8770.0,15-Apr-2009,1796.0,136000.0,Luxury,Available
4,4,4016,1026210,Honda Stream 1.8A X (COE till 09/2023),12800.0,7040.0,04-Sep-2008,1799.0,240000.0,MPV,Available


In [276]:
dealer_listings_df_clone.groupby(['dealer_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).head(n = 10)

,dealer_id,counts
421,2976,465
438,3028,405
154,2010,374
657,3597,362
488,3175,362
68,1168,326
566,3382,322
468,3113,315
568,3392,284
335,2690,251


In [277]:
pivoted = pd.pivot_table(dealer_listings_df_clone, values='listing_id', index=['dealer_id'], columns=['vehicle_type', 'status'], aggfunc='count', fill_value=0)

pivoted.columns = [f'{j}_{i}' for i, j in pivoted.columns]

dealer_by_type = pd.DataFrame(pivoted.to_records())

dealer_by_type.head()

,dealer_id,Available_Bus,CLOSED_Bus,SOLD_Bus,Available_Hatchback,CLOSED_Hatchback,SOLD_Hatchback,Available_Luxury,CLOSED_Luxury,SOLD_Luxury,Available_MPV,CLOSED_MPV,SOLD_MPV,Available_Others,SOLD_Others,Available_SUV,CLOSED_SUV,SOLD_SUV,Available_Sedan,CLOSED_Sedan,SOLD_Sedan,Available_Sports,CLOSED_Sports,SOLD_Sports,Available_Stationwagon,SOLD_Stationwagon,Available_Truck,CLOSED_Truck,SOLD_Truck,Available_Van,CLOSED_Van,SOLD_Van
0,1001,0,0,0,9,0,10,1,0,7,12,0,11,0,0,21,0,21,12,0,38,0,0,1,2,1,0,0,0,0,0,0
1,1002,0,0,0,2,0,2,0,0,0,2,0,0,0,0,2,0,0,4,0,2,0,0,0,3,0,0,0,0,0,0,1
2,1005,0,0,0,2,0,5,7,0,8,2,0,3,0,0,4,0,6,6,0,3,5,0,5,0,0,0,0,0,0,0,2
3,1009,0,0,0,12,0,8,11,0,15,9,0,10,0,0,12,0,14,9,0,8,2,0,2,0,2,0,0,0,0,0,0
4,1010,0,0,0,5,0,5,8,0,5,15,0,10,0,0,8,0,8,2,0,3,2,0,3,1,3,0,0,0,2,0,5


In [278]:
df_task1_train_prediction['discount'] = (df_task1_train_prediction['predicted_price'] - df_task1_train_prediction['price']) / df_task1_train_prediction['predicted_price']

df_task1_train_prediction.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price,predicted_price,dealer_id,discount
0,1030324,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013.0,NaN,09-dec-2013,luxury sedan,"parf car, premium ad car, low mileage car",auto,1560.0,135.0,NaN,1997.0,1.0,17700.0,77100.0,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,NaN,uncategorized,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",NaN,71300.0,97407.978598,2389,0.268027
1,1021510,Toyota Hiace 3.0M,NaN,hiace,high loan available! low mileage unit. wear an...,2014.0,NaN,26-jan-2015,van,premium ad car,manual,1740.0,NaN,diesel,2982.0,3.0,11630.0,10660.0,NaN,3648.0,110112.0,27502.0,1376.0,NaN,25-jan-2035,uncategorized,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,NaN,43800.0,64184.991920,2976,0.317597
2,1026909,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016.0,NaN,25-jul-2016,luxury sedan,"parf car, premium ad car",auto,1430.0,90.0,NaN,1595.0,1.0,15070.0,53694.0,740.0,44517.0,80000.0,27886.0,26041.0,NaN,NaN,uncategorized,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,NaN,95500.0,84798.727152,2217,-0.126196
3,1019371,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019.0,NaN,17-nov-2020,luxury sedan,"parf car, almost new car, consignment car",auto,1635.0,115.0,NaN,1497.0,1.0,16400.0,40690.0,684.0,80301.0,9800.0,46412.0,56977.0,NaN,NaN,uncategorized,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,NaN,197900.0,199098.750383,3734,0.006021
4,1031014,Honda Civic 1.6A VTi,NaN,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019.0,NaN,20-sep-2019,mid-sized sedan,parf car,auto,1237.0,92.0,NaN,1597.0,1.0,10450.0,26667.0,742.0,36453.0,40000.0,20072.0,20101.0,NaN,NaN,uncategorized,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",NaN,103200.0,92988.269468,2202,-0.109817


In [279]:
dealer_discount = df_task1_train_prediction[['dealer_id', 'discount']].groupby(['dealer_id'], group_keys=False)['discount'].agg(['mean', 'count']).sort_values(by='count', ascending=False)

dealer_discount.head()

,mean,count
dealer_id,,
<NA>,0.042987,3430
2976,0.066680,282
3175,0.036567,183
3028,0.074254,141
2690,0.040104,140


In [280]:
dealer_joined = dealer_by_type.set_index('dealer_id').join(dealer_discount).join(dealers_df.set_index('dealer_id'))
dealer_joined.head()

,Available_Bus,CLOSED_Bus,SOLD_Bus,Available_Hatchback,CLOSED_Hatchback,SOLD_Hatchback,Available_Luxury,CLOSED_Luxury,SOLD_Luxury,Available_MPV,CLOSED_MPV,SOLD_MPV,Available_Others,SOLD_Others,Available_SUV,CLOSED_SUV,SOLD_SUV,Available_Sedan,CLOSED_Sedan,SOLD_Sedan,Available_Sports,CLOSED_Sports,SOLD_Sports,Available_Stationwagon,SOLD_Stationwagon,Available_Truck,CLOSED_Truck,SOLD_Truck,Available_Van,CLOSED_Van,SOLD_Van,mean,count,dealer_name
dealer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0,0,0,9,0,10,1,0,7,12,0,11,0,0,21,0,21,12,0,38,0,0,1,2,1,0,0,0,0,0,0,0.028262,53.0,Lake View Credit Pte Ltd
1002,0,0,0,2,0,2,0,0,0,2,0,0,0,0,2,0,0,4,0,2,0,0,0,3,0,0,0,0,0,0,1,0.011269,3.0,Jia Yi Car Pte Ltd
1005,0,0,0,2,0,5,7,0,8,2,0,3,0,0,4,0,6,6,0,3,5,0,5,0,0,0,0,0,0,0,2,0.124305,26.0,Island Car Trading Pte Ltd
1009,0,0,0,12,0,8,11,0,15,9,0,10,0,0,12,0,14,9,0,8,2,0,2,0,2,0,0,0,0,0,0,0.017871,15.0,Thong Lee Trading Pte Ltd
1010,0,0,0,5,0,5,8,0,5,15,0,10,0,0,8,0,8,2,0,3,2,0,3,1,3,0,0,0,2,0,5,-0.068573,20.0,Tan Wei Auto Pte Ltd


In [281]:
dealer_score_by_type = dealer_joined.copy()


columns_to_drop = ['CLOSED_Bus','CLOSED_Hatchback','CLOSED_Luxury','CLOSED_MPV','CLOSED_Others','CLOSED_SUV','CLOSED_Sedan','CLOSED_Sports','CLOSED_Stationwagon','CLOSED_Truck','CLOSED_Van']

dealer_score_by_type = dealer_score_by_type.drop(columns_to_drop, axis=1, errors='ignore')


dealer_score_by_type.head()

,Available_Bus,SOLD_Bus,Available_Hatchback,SOLD_Hatchback,Available_Luxury,SOLD_Luxury,Available_MPV,SOLD_MPV,Available_Others,SOLD_Others,Available_SUV,SOLD_SUV,Available_Sedan,SOLD_Sedan,Available_Sports,SOLD_Sports,Available_Stationwagon,SOLD_Stationwagon,Available_Truck,SOLD_Truck,Available_Van,SOLD_Van,mean,count,dealer_name
dealer_id,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0,0,9,10,1,7,12,11,0,0,21,21,12,38,0,1,2,1,0,0,0,0,0.028262,53.0,Lake View Credit Pte Ltd
1002,0,0,2,2,0,0,2,0,0,0,2,0,4,2,0,0,3,0,0,0,0,1,0.011269,3.0,Jia Yi Car Pte Ltd
1005,0,0,2,5,7,8,2,3,0,0,4,6,6,3,5,5,0,0,0,0,0,2,0.124305,26.0,Island Car Trading Pte Ltd
1009,0,0,12,8,11,15,9,10,0,0,12,14,9,8,2,2,0,2,0,0,0,0,0.017871,15.0,Thong Lee Trading Pte Ltd
1010,0,0,5,5,8,5,15,10,0,0,8,8,2,3,2,3,1,3,0,0,2,5,-0.068573,20.0,Tan Wei Auto Pte Ltd


In [282]:
columns_to_scale = ['Available_Bus','Available_Hatchback','Available_Luxury','Available_MPV','Available_Others','Available_SUV','Available_Sedan',
                    'Available_Sports','Available_Stationwagon','Available_Truck','Available_Van',
                    'SOLD_Bus','SOLD_Hatchback','SOLD_Luxury','SOLD_MPV','SOLD_Others','SOLD_SUV','SOLD_Sedan','SOLD_Sports','SOLD_Stationwagon','SOLD_Truck','SOLD_Van',
                    'mean', 'count']

scaler = MinMaxScaler()

dealer_score_by_type[columns_to_scale] = scaler.fit_transform(dealer_score_by_type[columns_to_scale])

dealer_score_by_type.head()

,Available_Bus,SOLD_Bus,Available_Hatchback,SOLD_Hatchback,Available_Luxury,SOLD_Luxury,Available_MPV,SOLD_MPV,Available_Others,SOLD_Others,Available_SUV,SOLD_SUV,Available_Sedan,SOLD_Sedan,Available_Sports,SOLD_Sports,Available_Stationwagon,SOLD_Stationwagon,Available_Truck,SOLD_Truck,Available_Van,SOLD_Van,mean,count,dealer_name
dealer_id,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0.0,0.0,0.321429,0.250,0.02,0.111111,0.37500,0.333333,0.0,0.0,0.250000,0.256098,0.300,0.506667,0.000000,0.027027,0.250,0.111111,0.0,0.0,0.000000,0.000,0.608554,0.185053,Lake View Credit Pte Ltd
1002,0.0,0.0,0.071429,0.050,0.00,0.000000,0.06250,0.000000,0.0,0.0,0.023810,0.000000,0.100,0.026667,0.000000,0.000000,0.375,0.000000,0.0,0.0,0.000000,0.008,0.598563,0.007117,Jia Yi Car Pte Ltd
1005,0.0,0.0,0.071429,0.125,0.14,0.126984,0.06250,0.090909,0.0,0.0,0.047619,0.073171,0.150,0.040000,0.087719,0.135135,0.000,0.000000,0.0,0.0,0.000000,0.016,0.665026,0.088968,Island Car Trading Pte Ltd
1009,0.0,0.0,0.428571,0.200,0.22,0.238095,0.28125,0.303030,0.0,0.0,0.142857,0.170732,0.225,0.106667,0.035088,0.054054,0.000,0.222222,0.0,0.0,0.000000,0.000,0.602445,0.049822,Thong Lee Trading Pte Ltd
1010,0.0,0.0,0.178571,0.125,0.16,0.079365,0.46875,0.303030,0.0,0.0,0.095238,0.097561,0.050,0.040000,0.035088,0.081081,0.125,0.333333,0.0,0.0,0.014925,0.040,0.551617,0.067616,Tan Wei Auto Pte Ltd


In [283]:
vehicle_types = ['Bus','Hatchback','Luxury','MPV','Others','SUV','Sedan','Sports','Stationwagon','Truck','Van']

def get_score_for_vehicle_type(x, veh_type):
    if x['Available_' + veh_type] > 0.1:
        return x['Available_' + vehicle_type] * 3 + x['SOLD_' + vehicle_type] * 3 + x['mean'] * 3 + x['count']
    else:
        return 0

for vehicle_type in vehicle_types:
    dealer_score_by_type[vehicle_type + '_score'] = dealer_score_by_type.apply(lambda x: get_score_for_vehicle_type(x, vehicle_type), axis = 1)

dealer_score_by_type = dealer_score_by_type.drop(columns_to_scale, axis=1, errors='ignore')

dealer_score_by_type.to_csv('task_3_data/dealer_score_by_type.csv')
    
dealer_score_by_type.head()

,dealer_name,Bus_score,Hatchback_score,Luxury_score,MPV_score,Others_score,SUV_score,Sedan_score,Sports_score,Stationwagon_score,Truck_score,Van_score
dealer_id,,,,,,,,,,,,
1001,Lake View Credit Pte Ltd,0.0,3.725001,0.000000,4.135715,0.0,3.529008,4.430715,0.0,3.094049,0.0,0.0
1002,Jia Yi Car Pte Ltd,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2.927806,0.0,0.0
1005,Island Car Trading Pte Ltd,0.0,0.000000,2.884998,0.000000,0.0,0.000000,2.654046,0.0,0.000000,0.0,0.0
1009,Thong Lee Trading Pte Ltd,0.0,3.742870,3.231442,3.609997,0.0,2.797923,2.852156,0.0,0.000000,0.0,0.0
1010,Tan Wei Auto Pte Ltd,0.0,2.633181,2.440562,4.037808,0.0,0.000000,0.000000,0.0,3.097467,0.0,0.0


In [284]:
dealer_score_by_type.nlargest(10, 'Bus_score')

,dealer_name,Bus_score,Hatchback_score,Luxury_score,MPV_score,Others_score,SUV_score,Sedan_score,Sports_score,Stationwagon_score,Truck_score,Van_score
dealer_id,,,,,,,,,,,,
1060,Think One Automobile & Trading,7.879423,0.000000,0.000000,3.637945,0.0,0.000000,0.000000,0.0,2.254423,3.965510,4.041751
3113,ABS Bus Pte Ltd,4.701889,2.466550,2.700836,0.000000,0.0,0.000000,0.000000,0.0,0.000000,4.602948,6.629450
2976,Car (S) Pte Ltd,4.494935,3.589859,3.563907,5.305363,0.0,0.000000,4.118431,0.0,3.976764,8.110822,7.837431
1328,Hon Brothers Pte Ltd,4.113804,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,2.662437
2854,Pioneer Auto,3.311001,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3009,Yong Khiong Credit Pte Ltd,3.046975,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2723,Net Link Partners Pte Ltd,3.008897,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,4.408374,4.584730
1168,ABWIN (1994) Pte Ltd,2.800339,2.436617,0.000000,3.220302,0.0,0.000000,2.715188,0.0,2.448522,6.720188,4.430696
1099,COE Auto Trading,2.667164,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3.216481,2.646516


In [285]:
dealer_score_by_type.nlargest(10, 'Luxury_score')

,dealer_name,Bus_score,Hatchback_score,Luxury_score,MPV_score,Others_score,SUV_score,Sedan_score,Sports_score,Stationwagon_score,Truck_score,Van_score
dealer_id,,,,,,,,,,,,
3028,360 VR Cars,0.0,6.862154,7.833583,6.177738,0.000000,5.303095,6.450011,5.666747,4.446678,0.0,0.0
3175,Monster Motors Pte Ltd,0.0,6.141570,6.974665,4.618681,0.000000,4.977546,5.472999,8.163674,3.529665,0.0,0.0
3382,CarQuotz Singapore,0.0,7.511742,6.594599,5.830735,0.000000,4.047456,5.074599,0.000000,4.237932,0.0,0.0
2010,VINCAR Pte Ltd,0.0,3.972299,6.356823,6.846001,0.000000,7.021167,3.038728,5.249596,6.327061,0.0,0.0
1213,Republic Auto,0.0,5.495429,5.602571,4.625461,0.000000,4.382014,3.966143,0.000000,4.172810,0.0,0.0
1463,Premium Automobiles,0.0,3.842012,5.385108,0.000000,0.000000,3.560044,0.000000,0.000000,0.000000,0.0,0.0
2351,Performance Premium Selection Limited,0.0,5.508469,5.347279,5.378356,0.000000,5.304898,0.000000,0.000000,0.000000,0.0,0.0
3049,Motorist Pte Ltd,0.0,3.990064,5.329826,4.443311,0.000000,4.092067,3.456492,3.931656,2.511492,0.0,0.0
2563,Prem Roy Motoring,0.0,3.199455,4.815883,3.673075,4.181597,3.589263,3.786597,5.535794,3.223264,0.0,0.0


In [286]:
from joblib import dump, load


recommendation_model = load('recommendations_model.joblib') 

df_task1_train_cleaned = pd.read_csv('task_3_cleaned_training_data.csv')

In [287]:
vehicle_map = {
    'bus/mini bus': 'Bus',
    'hatchback': 'Hatchback',
    'luxury sedan': 'Luxury',
    'mpv': 'MPV',
    'others': 'Others',
    'suv': 'SUV',
    'mid-sized sedan': 'Sedan',
    'sports car': 'Sports',
    'stationwagon': 'Stationwagon',
    'truck': 'Truck',
    'van': 'Van'
}

In [288]:
def get_improved_top_recommendations_by_listing_id(listing_id):
    row_index = df_task1_train_prediction[df_task1_train_prediction['listing_id'] == listing_id].index.tolist()[0]
    vehicle_type = vehicle_map[df_task1_train_prediction.iloc[row_index]['type_of_vehicle']]
    d, i = recommendation_model.kneighbors([df_task1_train_cleaned.iloc[row_index]], n_neighbors=11)
    recommendation_df = df_task1_train_prediction.iloc[i[0]][1:]
    recommendation_df = recommendation_df.set_index('dealer_id').join(dealer_score_by_type[[vehicle_type + '_score', 'dealer_name']])
    recommendation_df = recommendation_df.sort_values(by=vehicle_type + '_score', ascending=False)
    return recommendation_df

In [289]:
get_improved_top_recommendations_by_listing_id(1021510)

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price,predicted_price,discount,Van_score,dealer_name
dealer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2976,1020689,Toyota Hiace 3.0M (COE till 05/2026),NaN,hiace,highly sought after van in the market. rare un...,2011.0,NaN,04-may-2011,van,"coe car, premium ad car",manual,2080.0,NaN,diesel,2982.0,3.0,8490.0,20261.0,NaN,19006.0,NaN,25820.0,1291.0,NaN,03-may-2031,uncategorized,rare unit hiace with chiller. view specs of th...,NaN,NaN,43800.0,35387.739393,-0.237717,7.837431,Car (S) Pte Ltd
2976,987868,Toyota Hiace 3.0M DX,toyota,hiace,high loan / low driveaway available! low milea...,2015.0,NaN,28-aug-2015,van,premium ad car,manual,1800.0,NaN,diesel,2982.0,2.0,10930.0,21179.0,NaN,8489.0,NaN,27936.0,1397.0,NaN,27-aug-2035,uncategorized,view specs of the toyota hiace,"front and back recording camera, reverse sensor.",NaN,48200.0,66674.154528,0.277081,7.837431,Car (S) Pte Ltd
2976,1025076,Mercedes-Benz Citan 109 CDI Van Extra-Long 2 S...,mercedes-benz,citan,3+1 premium warranty given! full servicing + i...,2016.0,NaN,27-jun-2016,van,premium ad car,manual,1420.0,NaN,diesel,1461.0,2.0,8850.0,18096.0,NaN,8755.0,66220.0,24294.0,1215.0,NaN,26-jun-2036,uncategorized,view specs of the mercedes-benz citan,NaN,NaN,47100.0,59820.131209,0.212640,7.837431,Car (S) Pte Ltd
3392,1017960,Toyota Hiace 3.0M (COE till 07/2026),NaN,hiace,"excellent condition, low mileage, well service...",2011.0,NaN,19-jul-2011,van,"coe car, premium ad car",manual,1780.0,NaN,diesel,2982.0,4.0,7920.0,19904.0,NaN,19500.0,NaN,25820.0,1291.0,NaN,18-jul-2031,uncategorized,view specs of the toyota hiace,NaN,NaN,42700.0,43126.476996,0.009889,6.957442,SG Motor Link Pte Ltd
3113,1013842,Nissan NV350 2.5M,nissan,nv350,nissan nv350 manual diesel. clean interior wit...,2016.0,NaN,27-sep-2016,van,premium ad car,manual,1800.0,NaN,diesel,2488.0,3.0,8800.0,4863.0,NaN,2475.0,NaN,23002.0,1151.0,NaN,26-sep-2036,uncategorized,view specs of the nissan nv350,NaN,NaN,49300.0,56162.283995,0.122187,6.629450,ABS Bus Pte Ltd
2723,986638,Toyota Hiace 3.0M (COE till 03/2026),NaN,hiace,excellent drive off condition. no repair neede...,2011.0,NaN,23-mar-2011,van,"coe car, premium ad car",manual,1780.0,NaN,diesel,2982.0,2.0,8260.0,19240.0,NaN,17606.0,174375.0,25820.0,1291.0,NaN,22-mar-2031,uncategorized,view specs of the toyota hiace,NaN,NaN,41600.0,32649.332784,-0.274145,4.584730,Net Link Partners Pte Ltd
3164,992874,Toyota Hiace 3.0M,NaN,hiace,"3 month warranty, fully serviced before handov...",2014.0,NaN,04-dec-2014,van,-,manual,1760.0,NaN,diesel,2982.0,3.0,12760.0,35240.0,NaN,11547.0,150302.0,29804.0,1491.0,NaN,03-dec-2034,uncategorized,view specs of the toyota hiace,NaN,NaN,46000.0,56604.809672,0.187348,3.246181,Skylink Auto Pte Ltd
3761,993969,Nissan NV200 1.5M,nissan,nv200,816,2016.0,NaN,14-nov-2016,van,premium ad car,manual,1260.0,NaN,diesel,1461.0,3.0,8020.0,24542.0,NaN,12816.0,NaN,20226.0,1012.0,NaN,13-nov-2036,uncategorized,view specs of the nissan nv200,NaN,NaN,46100.0,46940.453323,0.017905,0.000000,Dpex Motor Pte Ltd
1436,1027302,Nissan NV200 1.5M,nissan,nv200,new arrival nissan nv200 1.5m ! super popular ...,2014.0,NaN,08-may-2014,van,-,manual,1320.0,NaN,diesel,1461.0,2.0,15580.0,4999.0,NaN,1341.0,86120.0,19628.0,982.0,NaN,07-may-2034,uncategorized,view specs of the nissan nv200,NaN,NaN,46000.0,51120.491979,0.100165,NaN,NaN
